# Passo a passo para o algoritmo Random Forest:
## **1) Bootstrap:**
- Dado um conjunto de dados original de tamanho n
- Crie m novos conjuntos de dados (amostras bootstrap) de tamanho n
- Cada amostra é criada através de amostragem com reposição do conjunto original
## **2) Construção das árvores:**
- Para cada amostra bootstrap, construa uma árvore de decisão
- Ao construir cada nó da árvore:
- Selecione aleatoriamente k features (k < número total de features)
- Escolha a melhor feature entre as k para dividir o nó
- Cresça a árvore até a profundidade máxima ou até que um critério de parada seja atingido
## **3) Agregação:**
- Para fazer previsões, use todas as árvores construídas
- Para classificação: use votação majoritária
- Para regressão: calcule a média das previsões

## Explicação do Random Forest:

   Random Forest é um algoritmo de ensemble learning que combina múltiplas árvores de decisão para criar um modelo mais robusto e preciso. Imagine que você está tentando tomar uma decisão importante e pede conselhos a um grupo de amigos. Cada amigo tem acesso a informações ligeiramente diferentes (as amostras bootstrap) e considera apenas alguns aspectos aleatórios do problema de cada vez (a seleção aleatória de features). No final, você considera todos os conselhos e toma uma decisão baseada na opinião majoritária. Isso é essencialmente o que o Random Forest faz.

##   **O "random" em Random Forest vem de dois elementos de aleatoriedade:**
 - 1) A criação de amostras bootstrap aleatórias para treinar cada árvore.
 - 2) A seleção aleatória de um subconjunto de features em cada divisão da
   árvore.

  Essa aleatoriedade ajuda a criar um conjunto diversificado de árvores, cada uma capturando diferentes padrões nos dados. Ao combinar essas árvores, o Random Forest pode produzir previsões mais estáveis e precisas, além de ser menos propenso ao overfitting em comparação com uma única árvore de decisão.

# Diferenças entre Bagging e Random Forest:
## 1. **Tipo de modelo base:**
- Bagging: Pode usar qualquer tipo de modelo como base (árvores de decisão, regressão linear, etc.).
- Random Forest: Usa especificamente árvores de decisão como modelos base.
## 2. **Seleção de features:**
- Bagging: Usa todas as features disponíveis para cada modelo base.
- Random Forest: Seleciona aleatoriamente um subconjunto de features em cada divisão da árvore.
## 3. **Diversidade dos modelos:**
- Bagging: A diversidade vem principalmente das diferentes amostras bootstrap.
- Random Forest: A diversidade vem tanto das amostras bootstrap quanto da seleção aleatória de features.
## **4. Correlação entre modelos:**
- Bagging: Os modelos podem ser altamente correlacionados se usarem todas as features.
- Random Forest: A seleção aleatória de features reduz a correlação entre as árvores.
## **5. Interpretabilidade:**
- Bagging: A interpretabilidade depende do modelo base escolhido.
- Random Forest: Oferece medidas de importância de features, facilitando a interpretação.
## **6. Overfitting:**
Bagging: Reduz o overfitting, mas pode ainda ser suscetível se os modelos base forem complexos.
Random Forest: Geralmente mais resistente ao overfitting devido à seleção aleatória de features.
## **7. Aplicação:**
Bagging: Mais genérico, pode ser aplicado a uma variedade de problemas e modelos.
Random Forest: Específico para problemas onde árvores de decisão são apropriadas.

In [1]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

In [2]:
class SimpleRandomForest:
    def __init__(self, n_estimators=10, max_features='sqrt', max_depth=None):
        self.n_estimators = n_estimators
        self.max_features = max_features
        self.max_depth = max_depth
        self.trees = []

    def bootstrap_sample(self, X, y):
        n_samples = X.shape[0]
        idxs = np.random.choice(n_samples, size=n_samples, replace=True)
        return X[idxs], y[idxs]

    def get_random_features(self, X):
        n_features = X.shape[1]
        if self.max_features == 'sqrt':
            n_random_features = int(np.sqrt(n_features))
        elif self.max_features == 'log2':
            n_random_features = int(np.log2(n_features))
        else:
            n_random_features = self.max_features
        random_feature_idxs = np.random.choice(n_features, size=n_random_features, replace=False)
        return random_feature_idxs

    def fit(self, X, y):
        self.trees = []
        for _ in range(self.n_estimators):
            tree = DecisionTreeClassifier(max_features=self.max_features, max_depth=self.max_depth)
            X_sample, y_sample = self.bootstrap_sample(X, y)
            tree.fit(X_sample, y_sample)
            self.trees.append(tree)

    def predict(self, X):
        tree_preds = np.array([tree.predict(X) for tree in self.trees])
        return np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=0, arr=tree_preds)

In [3]:
# Exemplo de uso
X, y = make_classification(n_samples=1000, n_features=20, n_classes=2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rf = SimpleRandomForest(n_estimators=100, max_features='sqrt', max_depth=10)
rf.fit(X_train, y_train)

predictions = rf.predict(X_test)
accuracy = np.mean(predictions == y_test)
print(f"Acurácia: {accuracy:.2f}")

Acurácia: 0.89
